In [1]:
import cv2

In [2]:
from cvzone.HandTrackingModule import HandDetector

In [22]:
from tensorflow.keras.preprocessing import image
import numpy as np
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8, maxHands=2)
 
while True:
    success, img = cap.read()
    img=cv2.flip(img, 1)
#     hands, img = detector.findHands(img)  # With Draw
    hands = detector.findHands(img, draw=False)  # No Draw
 
    if hands:
        # Hand 1
        hand1 = hands[0]
        
        bbox1 = hand1["bbox"]  # Bounding Box info x,y,w,h
        
        centerPoint1 = hand1["center"]  # center of the hand cx,cy
        handType1 = hand1["type"]  # Hand Type Left or Right
 
        # print(len(lmList1),lmList1)
        # print(bbox1)
        # print(centerPoint1)
        fingers1 = detector.fingersUp(hand1)
        #length, info, img = detector.findDistance(lmList1&#91;8], lmList1&#91;12], img) # with draw
        #length, info = detector.findDistance(lmList1&#91;8], lmList1&#91;12])  # no draw
 
        bbox=bbox1
        if len(hands) == 2:
            hand2 = hands[1]
            
            bbox2 = hand2["bbox"]  # Bounding Box info x,y,w,h
            
            centerPoint2 = hand2["center"]  # center of the hand cx,cy
            handType2 = hand2["type"]  # Hand Type Left or Right
 
            fingers2 = detector.fingersUp(hand2)
            # print(fingers1, fingers2)
            #length, info, img = detector.findDistance(lmList1&#91;8], lmList2&#91;8], img) # with draw
            bbox=[min(bbox1[0], bbox2[0]), min(bbox1[1],bbox2[1]), bbox1[2]+bbox2[2]+20, max(bbox1[3],bbox2[3])+20]
        startx=0
        starty=0
        endx=img.shape[1]
        endy=img.shape[0]
        
        if bbox[1] - 20>=0:
            starty=bbox[1] - 20
        if bbox[1] + bbox[3] + 20<endy:
            endy=bbox[1] + bbox[3] + 20
        if bbox[0] - 20>=0:
            startx=bbox[0] - 20
        if bbox[0] + bbox[2] + 20<endx:
            endx=bbox[0] + bbox[2] + 20
        
        
        crop_img = img[starty:endy, startx: endx]
        cv2.rectangle(img, (bbox[0] - 20, bbox[1] - 20), (bbox[0] + bbox[2] + 20, bbox[1] + bbox[3] + 20), (0,255,0),3)
       
#         print(crop_img.shape)
        
        # res=cv2.flip(res,1)
        out=image_preprocessing_and_prediction(crop_img)
        
        cv2.putText(img, out, (bbox[0] - 20, bbox[1] - 20), cv2.FONT_HERSHEY_COMPLEX, 3, (0,0,255), 10)
        
        

    # applying gaussian blur
#         value = (35, 35)
#         blurred = cv2.GaussianBlur(grey, value, 0)

#         # thresholdin: Otsu's Binarization method
#         _, thresh1 = cv2.threshold(blurred, 127, 255,
#                                    cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

#         # show thresholded image
#         cv2.imshow('Thresholded1', thresh1)
        
        
        
        
#         
            
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    cv2.imshow("Image", img)
#     if hands:
#         cv2.imshow("Frame", crop_img)
    
    cv2.waitKey(1)
cv2.destroyAllWindows()
# vs.stream.release()
cap.release()



In [18]:
def image_preprocessing_and_prediction(res):

  res=cv2.resize(res,(32,32))
  # res=cv2.cvtColor(res, cv2.COLOR_GRAY2RGB)
  test_image = image.img_to_array(res)
  test_image = np.expand_dims(test_image, axis = 0)
  test_image=test_image.astype('float32')/255.0
  result = loaded_model.predict(test_image)
  x=result.tolist()[0]
  indexes=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K','L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V','W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']
  return indexes[x.index(max(x))]
  

In [11]:
frame = cv2.imread("1.jpeg")
frame2=cv2.resize(frame,(32,32))
test_image = image.img_to_array(frame2)
print(test_image.shape)
test_image = np.expand_dims(test_image, axis = 0)
print(test_image.shape)
# test_image = test_image.astype('float32')

test_image=test_image.astype('float32')/255.0

result = loaded_model.predict(test_image)

(32, 32, 3)
(1, 32, 32, 3)


In [13]:
x=result.tolist()[0]
indexes=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K','L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V','W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']
print(indexes[x.index(max(x))])


U


In [4]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image

In [3]:
import os

from keras.models import model_from_json
json_file = open("model-bw-asl.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("model-bw-asl.h5")